## Lab 02: Integration with LangChain

### We can use Actions to integrate LangChain.


In [1]:
## setup OPENAI_API_KEY

import os
from pprint import pp
import getpass

if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass.getpass('Enter your OpenAI API key:')


We can also use variables in combination with if/else statements to make the behavior more dynamic

In [3]:
from nemoguardrails import LLMRails, RailsConfig

In [4]:
yaml_content = """
models:
- type: main
  engine: openai
  model: text-davinci-003
# - type: embeddings
#   engine: openai
#   model: text-embedding-ada-002
"""

In [5]:
colang_content = """
define user express greeting
    "hello"
    "hi"

define bot express greeting
    "Hello there!! Can I help you today?"

define bot personal greeting
    "Hello $username, nice to see you again!"

define flow hello
    user express greeting
    if $username
      bot personal greeting
    else
        bot express greeting

define flow
    user ...
    $answer = execute qa_chain(query=$last_user_message)
    bot $answer
"""

config = RailsConfig.from_content(
  	yaml_content=yaml_content,
    colang_content=colang_content
)
rails = LLMRails(config=config)

/Users/jimmy.liao/.pyenv/versions/3.10.13/envs/guardrails/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# vectorstore = Chroma.from_texts(texts=["CatBank's Furry Paws Credit Card offers 2% cashback on pet stores and 1% on all other purchases.",
#                                  "Meow Loans at CatBank come with a competitive 5% APR, subject to your catnip credit score.",
#                                  "Earn up to 4% interest with CatBank's Kitty Savers account, with a minimum deposit of $500.",
#                                  "Invest in the Whisker Growth Fund with a minimum of $1,000 and enjoy purr-tential annual returns of 7%.",
#                                  "Open a Feline Secure savings account with just $100; enjoy up to 3% interest with no monthly fees."],
#                                 embedding=OpenAIEmbeddings())
vectorstore = Chroma.from_texts(texts=["CatBank's Furry Paws Credit Card offers 2% cashback on pet stores and 1% on all other purchases.",
                                 "Meow Loans at CatBank come with a competitive 5% APR, subject to your catnip credit score.",
                                 "Earn up to 4% interest with CatBank's Kitty Savers account, with a minimum deposit of $500.",
                                 "Invest in the Whisker Growth Fund with a minimum of $1,000 and enjoy purr-tential annual returns of 7%.",
                                 "Open a Feline Secure savings account with just $100; enjoy up to 3% interest with no monthly fees."],
                                embedding=OpenAIEmbeddings()
                                )

retriever = vectorstore.as_retriever()
prompt_template = """You are a helpful bot for our Bank. Only answer if your have got content in "Context".
Otherwise tell the user in a friendly way that you do not know and can not help with that.

Context: {context}

Question: {question}
Answer here:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(
    llm=rails.llm, # Use llm attached to rail instance
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
)

ValueError: You must provide an embedding function to compute embeddings.https://docs.trychroma.com/embeddings